## 8.4.1 勇敢面对还是知难而退

## 8.4.2 元类模型

In [2]:
# 我们可以通过type来直接创建一个类
import typing_extensions


class Eggs: pass
class Spam(Eggs):
    data = 1
    def meth(self, arg):
        return self.data + arg

# 我们使用type来创建一个和上面Spam一样的类
Spam2 = type('Spam2',(Eggs,),{'data':1, 'meth': (lambda x,y: x.data+y)})
s = Spam2()
print(s.data)
print(s.meth(4))

1
5


## 8.4.3 声明元类

In [ ]:
# 元类可以这样去声明
class Spam(metaclass=None): pass

## 8.4.4 编写元类

In [3]:
# 下面我们来创建一个元类
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('in metaone.new:', meta,classname,supers,classdict,sep='\n')
        return type.__new__(meta, classname, supers, classdict)

In [4]:
# 然后我们可以这样使用
class Eggs:
    pass
print('making class....')
class Spam(Eggs, metaclass=MetaOne):
    data = 1
    def meth(self, arg):
        return self.data + arg
print('making instance....')
x=Spam()
print('data:', x.data, x.meth(2))

making class....
in metaone.new:
<class '__main__.MetaOne'>
Spam
(<class '__main__.Eggs'>,)
{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7D678>}
making instance....
data: 1 3


In [5]:
# 元类还可以接入init，new创建并返回了类对象，然后init初始化了一个参数被传入的已经创建了的类
class MetaTwo(type):
    def __new__(meta, classname, supers, classdict):
        print('in metaTwo new:', meta,classname,supers,classdict,sep='\n')
        return type.__new__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('in metaTwo init:', Class,classname,supers,classdict,sep='\n')
        print('...init class object:', list(Class.__dict__.keys()))

In [6]:
class Eggs:
    pass
print('making class....')
class Spam(Eggs, metaclass=MetaTwo):
    data = 1
    def meth(self, arg):
        return self.data + arg
print('making instance....')
x=Spam()
print('data:', x.data, x.meth(2))

making class....
in metaTwo new:
<class '__main__.MetaTwo'>
Spam
(<class '__main__.Eggs'>,)
{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7D4C8>}
in metaTwo init:
<class '__main__.Spam'>
Spam
(<class '__main__.Eggs'>,)
{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7D4C8>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance....
data: 1 3


In [7]:
# 元类不一定是类，也可以是一个函数
def MetaFunc(classname,suupers,clasdict):
    print('in meta func', classname, suupers, clasdict,sep='\n..')
    return type(classname, suupers, clasdict)


In [8]:
class Eggs:
    pass
print('making class....')
class Spam(Eggs, metaclass=MetaFunc):
    data = 1
    def meth(self, arg):
        return self.data + arg
print('making instance....')
x=Spam()
print('data:', x.data, x.meth(2))

making class....
in meta func
..Spam
..(<class '__main__.Eggs'>,)
..{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD535E8>}
making instance....
data: 1 3


In [11]:
# 下面演示一下元类的重载
class SuperMeta(type):
    def __call__(meta, classname, supers, classdict):
        print('in supermeta call', classname, supers, classdict, sep='\n..')
        return type.__call__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('in supermeta init', classname, supers, classdict, sep='\n..')
        print("...init class object:", list(Class.__dict__.keys()))
print('making meta class')
class SubMeta(type, metaclass=SuperMeta):
    def __new__(meta, classname, supers, classdict):
        print('in subMeta bew', classname, supers, classdict, sep='\n..')
        return type.__new__(meta, classname, supers, classdict)
    def __init__(Class, classname, supers, classdict):
        print('in SubMeta init', classname, supers, classdict, sep='\n..')
        print("...init class object:", list(Class.__dict__.keys()))

making meta class
in supermeta init
..SubMeta
..(<class 'type'>,)
..{'__module__': '__main__', '__qualname__': 'SubMeta', '__new__': <function SubMeta.__new__ at 0x000002156BD7DEE8>, '__init__': <function SubMeta.__init__ at 0x000002156BD7DCA8>}
...init class object: ['__module__', '__new__', '__init__', '__doc__']


In [12]:
class Eggs:
    pass
print('making class....')
class Spam(Eggs, metaclass=SubMeta):
    data = 1
    def meth(self, arg):
        return self.data + arg
print('making instance....')
x=Spam()
print('data:', x.data, x.meth(2))

making class....
in supermeta call
..Spam
..(<class '__main__.Eggs'>,)
..{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7F5E8>}
in subMeta bew
..Spam
..(<class '__main__.Eggs'>,)
..{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7F5E8>}
in SubMeta init
..Spam
..(<class '__main__.Eggs'>,)
..{'__module__': '__main__', '__qualname__': 'Spam', 'data': 1, 'meth': <function Spam.meth at 0x000002156BD7F5E8>}
...init class object: ['__module__', 'data', 'meth', '__doc__']
making instance....
data: 1 3


## 8.4.5 继承与实例

In [13]:
class MetaOne(type):
    def __new__(meta, classname, supers, classdict):
        print('in MetaOne new', classname)
        return type.__new__(meta, classname, supers, classdict)
    def toast(self):
        return 'toast'
class Super(metaclass=MetaOne):
    def spam(self):
        return 'spam'
class Sub(Super):
    def eggs(self):
        return 'eggs'

in MetaOne new Super
in MetaOne new Sub


In [14]:
x = Sub()
# 正常的类继承是可以使用的
print(x.eggs())
print(x.spam())

eggs
spam


In [15]:
# 但是不能使用元类的方法
print(x.toast())

AttributeError: 'Sub' object has no attribute 'toast'

In [16]:
# 但是类可以继承属性
print(Sub.eggs(x))
print(Sub.spam(x))
# 类可以使用原类的属性
print(Sub.toast())

eggs
spam
toast


## 8.4.6 元类方法

In [17]:
class A(type):
    def x(cls): print('ax', cls)
    def y(cls): print('ay', cls)
class B(metaclass=A):
    def y(self): print('by', self)
    def z(self): print('bz', self)

In [18]:
# 可以看到类继承的元类方法
print(B.x)
print(B.y)
print(B.z)
print(B.x())

<bound method A.x of <class '__main__.B'>>
<function B.y at 0x000002156BD82DC8>
<function B.z at 0x000002156BE413A8>
ax <class '__main__.B'>
None


In [19]:
I = B()
I.y()

by <__main__.B object at 0x000002156BE43748>


In [20]:
I.z()
# 实例化的对象无法获取元类方法
I.x()

bz <__main__.B object at 0x000002156BE43748>


AttributeError: 'B' object has no attribute 'x'

In [21]:
# 元类方法的运算符重载
class A(type):
    def __getitem__(cls, i):
        return cls.data[i]
class B(metaclass=A):
    data = 'spam'

In [22]:
print(B[0])
print(B.__getitem__)

s
<bound method A.__getitem__ of <class '__main__.B'>>


In [23]:
I = B()
print(I.data, B.data)

spam spam


In [24]:
# 对象也无法获取元类的重载方法
I[0]

TypeError: 'B' object is not subscriptable

In [25]:
# 我们在元类中定义一个getattr方法
class A(type):
    def __getattr__(cls, name):
        return getattr(cls.data, name)
class B(metaclass=A):
    data = 'spam'
print(B.upper())
print(B.upper)
print(B.__getattr__)

SPAM
<built-in method upper of str object at 0x000002156BD280B0>
<bound method A.__getattr__ of <class '__main__.B'>>


In [26]:
I = B()
I.upper
I.__getattr__

AttributeError: 'B' object has no attribute 'upper'

## 8.4.7 向类添加方法

In [27]:
# 我们可以通过元类，自动给子类添加方法
def eggsfunc(obj):
    return obj.value *4
def hamfunc(obj, value):
    return value+'ham'
class Extender(type):
    def __new__(meta, classname, supers, classdict):
        classdict['eggs'] = eggsfunc
        classdict['ham'] = hamfunc
        return type.__new__(meta, classname, supers, classdict)
class Client1(metaclass=Extender):
    def __init__(self,value):
        self.value = value
    def spam(self):
        return self.value * 2
class Client2(metaclass=Extender):
    value = 'ni?'

In [28]:
x = Client1('Ni!')
print(x.spam())
print(x.eggs())
print(x.ham('bacon'))

Ni!Ni!
Ni!Ni!Ni!Ni!
baconham


In [29]:
y = Client2()
print(y.eggs())
print(y.ham('aaa'))

ni?ni?ni?ni?
aaaham


In [30]:
# 实际上我们也可以通过装饰器来实现上面的效果
def eggsfunc(obj):
    return obj.value *4
def hamfunc(obj, value):
    return value+'ham'
def Extender(aClass):
    aClass.eggs = eggsfunc
    aClass.ham = hamfunc
    return aClass

In [31]:
@Extender
class Client3:
    def __init__(self, value):
        self.value = value
    def spam(self):
        return self.value * 2
@Extender
class Client4:
    value = 'ni?'
x = Client3('Ni!')
print(x.spam())
print(x.eggs())
print(x.ham('bacon'))
y = Client4()
print(y.eggs())
print(y.ham('sss'))

Ni!Ni!
Ni!Ni!Ni!Ni!
baconham
ni?ni?ni?ni?
sssham


类装饰器可以扮演和元类一样的角色

![](.8_images/c6a44007.png)

## 8.4.8 对方法应用装饰器